In [2]:
import pickle
import os
import sys

# Get the absolute path of the project root directory
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

if project_root not in sys.path:
    sys.path.append(project_root)

# Define the directory and file path
save_dir = "../data/parsed/"
load_path = os.path.join(save_dir, "builder.pkl")

# Load the builder object
with open(load_path, "rb") as f:
    builder = pickle.load(f)

In [7]:
sample = builder.run(builder.timestamps[0])
sample

This pandapower network includes the following parameter tables:
   - bus (118 elements)
   - load (91 elements)
   - gen (321 elements)
   - ext_grid (1 element)
   - line (177 elements)
   - trafo (9 elements)
   - bus_geodata (118 elements)
 and the following results tables:
   - res_bus (118 elements)
   - res_line (177 elements)
   - res_trafo (9 elements)
   - res_ext_grid (1 element)
   - res_load (91 elements)
   - res_gen (321 elements)

In [16]:
import pandas as pd

for attr in dir(sample):
    if isinstance(getattr(sample, attr), pd.DataFrame):  # Check if it's a DataFrame
        print(f"🔹 {attr}.head():")
        print(getattr(sample, attr).head())  # Show first few rows
        print("\n" + "-"*80 + "\n")  # Separator

🔹 _empty_res_asymmetric_load.head():
Empty DataFrame
Columns: [p_mw, q_mvar]
Index: []

--------------------------------------------------------------------------------

🔹 _empty_res_asymmetric_load_3ph.head():
Empty DataFrame
Columns: [p_a_mw, q_a_mvar, p_b_mw, q_b_mvar, p_c_mw, q_c_mvar]
Index: []

--------------------------------------------------------------------------------

🔹 _empty_res_asymmetric_sgen.head():
Empty DataFrame
Columns: [p_mw, q_mvar]
Index: []

--------------------------------------------------------------------------------

🔹 _empty_res_asymmetric_sgen_3ph.head():
Empty DataFrame
Columns: [p_a_mw, q_a_mvar, p_b_mw, q_b_mvar, p_c_mw, q_c_mvar]
Index: []

--------------------------------------------------------------------------------

🔹 _empty_res_bus.head():
Empty DataFrame
Columns: [vm_pu, va_degree, p_mw, q_mvar]
Index: []

--------------------------------------------------------------------------------

🔹 _empty_res_bus_3ph.head():
Empty DataFrame
Columns: [v

In [17]:
import networkx as nx
import torch
import torch.nn as nn
import torch_geometric.nn as pyg_nn
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx

def construct_nrel118_graph_sequence(builder, num_timesteps=10):
    """
    Constructs a sequence of PyTorch Geometric Data objects for the NREL 118-bus network.
    
    Args:
        builder: PandaPowerFlowBuilder instance
        num_timesteps: Number of timesteps to process
    
    Returns:
        List of PyG Data objects representing the power network state at each timestep
    """
    graph_sequence = []
    
    for t in range(min(num_timesteps, len(builder.timestamps))):
        sample = builder.run(builder.timestamps[t])
        G = nx.Graph()
        
        # Add nodes (buses) with their features
        for idx, bus in sample.bus.iterrows():
            res_bus = sample.res_bus.loc[idx] if idx in sample.res_bus.index else None
            G.add_node(idx, 
                      vn_kv=float(bus['vn_kv']),
                      min_vm_pu=float(bus['min_vm_pu']),
                      max_vm_pu=float(bus['max_vm_pu']),
                      type=bus['type'],
                      zone=bus['zone'],
                      vm_pu=float(res_bus['vm_pu']) if res_bus is not None else 1.0,
                      va_degree=float(res_bus['va_degree']) if res_bus is not None else 0.0,
                      p_mw=float(res_bus['p_mw']) if res_bus is not None else 0.0,
                      q_mvar=float(res_bus['q_mvar']) if res_bus is not None else 0.0)
        
        # Add edges (transmission lines)
        for idx, line in sample.line.iterrows():
            res_line = sample.res_line.loc[idx] if idx in sample.res_line.index else None
            G.add_edge(line['from_bus'], line['to_bus'],
                      r_ohm_per_km=float(line['r_ohm_per_km']),
                      x_ohm_per_km=float(line['x_ohm_per_km']),
                      length_km=float(line['length_km']),
                      p_from=float(res_line['p_from_mw']) if res_line is not None else 0.0,
                      q_from=float(res_line['q_from_mvar']) if res_line is not None else 0.0,
                      p_to=float(res_line['p_to_mw']) if res_line is not None else 0.0,
                      q_to=float(res_line['q_to_mvar']) if res_line is not None else 0.0,
                      loading=float(res_line['loading_percent']) if res_line is not None else 0.0)

        # Add transformers as edges
        for idx, trafo in sample.trafo.iterrows():
            res_trafo = sample.res_trafo.loc[idx] if idx in sample.res_trafo.index else None
            G.add_edge(trafo['hv_bus'], trafo['lv_bus'],
                      p_from=float(res_trafo['p_hv_mw']) if res_trafo is not None else 0.0,
                      q_from=float(res_trafo['q_hv_mvar']) if res_trafo is not None else 0.0,
                      p_to=float(res_trafo['p_lv_mw']) if res_trafo is not None else 0.0,
                      q_to=float(res_trafo['q_lv_mvar']) if res_trafo is not None else 0.0,
                      loading=float(res_trafo['loading_percent']) if res_trafo is not None else 0.0)

        # Add generators as nodes
        for idx, gen in sample.gen.iterrows():
            res_gen = sample.res_gen.loc[idx] if idx in sample.res_gen.index else None
            bus_idx = gen['bus']
            if bus_idx in G.nodes:
                G.nodes[bus_idx]['gen_p_mw'] = float(res_gen['p_mw']) if res_gen is not None else 0.0
                G.nodes[bus_idx]['gen_q_mvar'] = float(res_gen['q_mvar']) if res_gen is not None else 0.0

        # Convert to PyG Data
        data = from_networkx(G)
        
        # Extract node features
        node_features = torch.tensor([[G.nodes[i]['vn_kv'], 
                                       G.nodes[i]['min_vm_pu'],
                                       G.nodes[i]['max_vm_pu'],
                                       G.nodes[i]['vm_pu'], 
                                       G.nodes[i]['va_degree'],
                                       G.nodes[i]['p_mw'], 
                                       G.nodes[i]['q_mvar']] 
                                      for i in G.nodes], 
                                     dtype=torch.float)
        
        # Ensure bidirectional edges match edge_index.shape[1]
        edge_features_list = []
        for u, v, d in G.edges(data=True):
            edge_features_list.append([d['r_ohm_per_km'], d['x_ohm_per_km'], d['p_from'], d['q_from'], d['p_to'], d['q_to'], d['loading']])
            edge_features_list.append([d['r_ohm_per_km'], d['x_ohm_per_km'], d['p_to'], d['q_to'], d['p_from'], d['q_from'], d['loading']])  # Duplicate

        edge_features = torch.tensor(edge_features_list, dtype=torch.float)
        
        data.x = node_features
        data.edge_attr = edge_features
        
        graph_sequence.append(data)
    
    return graph_sequence

In [18]:
graph_sequences = construct_nrel118_graph_sequence(builder,10)

ValueError: Not all nodes contain the same attributes